[![View on GitHub](https://img.shields.io/badge/GitHub-View_on_GitHub-blue?logo=GitHub)](https://github.com/khuyentran1401/Data-science/blob/master/data_science_tools/pyLDAvis/preprocess.ipynb)

[<img src="https://deepnote.com/buttons/launch-in-deepnote.svg">](https://deepnote.com/project/Data-science-hxlyJpi-QrKFJziQgoMSmQ/%2FData-science%2Fdata_science_tools%2FpyLDAvis%2Fpreprocess.ipynb)

 [![View Article](https://img.shields.io/badge/View%20Article-View%20Full%20Article-red)](https://neptune.ai/blog/pyldavis-topic-modelling-exploration-tool-that-every-nlp-data-scientist-should-know)

In [ ]:
!pip install emoji==1.6.0 pyLDAvis==3.3.1 chart_studio==1.1.0 datapane==0.12.3

In [ ]:
!python -m spacy download en_core_web_sm

In [ ]:
%load_ext watermark
%watermark --iversions

The watermark extension is already loaded. To reload it, use:
  %reload_ext watermark
json        : 2.0.9
requests    : 2.26.0
re          : 2.2.1
pyLDAvis    : 3.3.1
chart_studio: 1.1.0
datapane    : 0.12.3
plotly      : 5.3.1
nltk        : 3.6.3
spacy       : 3.1.3
numpy       : 1.21.2
gensim      : 4.1.2
regex       : 2021.9.30
csv         : 1.0
matplotlib  : 3.4.3
emoji       : 1.6.0
pandas      : 1.2.5
psycopg2    : 2.9.1 (dt dec pq3 ext lo64)
argparse    : 1.1
seaborn     : 0.11.2
sklearn     : 0.0



# Import libaries

In [ ]:
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import numpy as np
import re

import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import string
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer

import csv
import argparse
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_recall_fscore_support, roc_auc_score
from sklearn import metrics, preprocessing
import numpy as np


# Required Libraries

#Base and Cleaning 
import json
import requests
import pandas as pd
import numpy as np
import emoji
import regex
import re
import string
from collections import Counter

#Visualizations
import plotly.express as px
import seaborn as sns
import matplotlib.pyplot as plt 
import pyLDAvis.gensim_models
import chart_studio
import chart_studio.plotly as py 
import chart_studio.tools as tls

#Natural Language Processing (NLP)
import spacy
import gensim
from spacy.tokenizer import Tokenizer
import gensim
import gensim.corpora as corpora
from gensim.corpora import Dictionary
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
from gensim.parsing.preprocessing import STOPWORDS as SW
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from pprint import pprint
from wordcloud import STOPWORDS
stopwords = set(STOPWORDS)
import datapane as dp 

import pickle 

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2021-10-12 02:44:11.382941: W tensorflow/stream_executor/platform/default/dso_loader.cc:60] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2021-10-12 02:44:11.382979: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/shared-libs/python3.7/py/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/root/venv/lib/python3.7/site-packages/datapane/client/api/report/core.py:119: DeprecationWarning: 'contextfunction' is renamed

# Get Data

In [ ]:
tweets = pd.read_csv('/datasets/tweets/dp-export-8927.csv') 

tweets = tweets.Tweets.values.tolist()

In [ ]:
tweets[0]

'Spoke at @AllHealthReform event on health care reform this morning. No live tweeting allowed. Did it really happen? @ddiamond True. You can cover \'more\' with \'more\' for \'less.\' But you don\'t want to be one of the \'less\' who get \'le… https://t.co/Ui54o2m5LD The hard part was not gawking at the Chrysler Building next door. My son built a 6 ft replica of it in our backyard. https://t.co/0aftXVUR3P @nicholas_bagley @onceuponA The other key question is which of these would hit Plan Year \'17 and which would hit Plan Year \'18. Must read from @nicholas_bagley and @onceuponA and must ask if the goal is orderly transition or disruption.\n\nhttps://t.co/PmAUOeMo6x @squelchedalot 11 generations all in VA in my family. Can\'t speak to all the branches though. #Obamacare Executive Order is not a specific action. It is a declaration of administrative war. Orderly transition or massive disruption? The President goes by. #InaugurationDay. https://t.co/RDQSx1VWq4 Motorcade in the parade. #I

# Preprocessing

In [ ]:
replacement_patterns = [
    #match url (i.e: https://t.co/5tF5G9VKtq)
    (r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', ''),

    #match user (i.e: @cerpintor )
    (r'@\w+', ''),

    #match hashtag (i.e: #WomensMarchOnWashington)
    (r'#\w+', ''),

    #Replace "&..." with ''
    (r'&\w+', '')
]

class RegexReplacer(object):
    def __init__(self, patterns = replacement_patterns):
        self.patterns = [(re.compile(regrex),repl) for (regrex, repl) in
                        patterns]
    
    #Replace the words that match the patterns with replacement words
    def replace(self, text):
        s = text
        for (pattern, repl) in self.patterns:
            s = re.sub(pattern, repl, s)
        return s

In [ ]:
def give_emoji_free_text(text):
    """
    Removes emoji's from tweets
    Accepts:
        Text (tweets)
    Returns:
        Text (emoji free tweets)
    """
    emoji_list = [c for c in text if c in emoji.UNICODE_EMOJI]
    clean_text = ' '.join([str for str in text.split() if not any(i in str for i in emoji_list)])
    return clean_text



In [ ]:
def lemmatization(text, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    
    doc = nlp(" ".join(text)) 
    return [token.lemma_ for token in doc if token.pos_ in allowed_postags]


In [ ]:
nlp = spacy.load("en_core_web_sm", disable=['parser', 'ner'])
tknz = TweetTokenizer()
replacer = RegexReplacer()
# Custom stopwords
custom_stopwords = ['hi','\n','\n\n', '&amp;', ' ', '.', '-', 'got', "it's", 'it’s', "i'm", 'i’m', 'im', 'want', 'like', '$', '@']
# Customize stop words by adding to the default list
STOP_WORDS = nlp.Defaults.stop_words.union(custom_stopwords)
# ALL_STOP_WORDS = spacy + gensim + wordcloud
ALL_STOP_WORDS = STOP_WORDS.union(SW).union(stopwords)

punc = string.punctuation

In [ ]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(tweets, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[tweets], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)


In [ ]:
def normalize(doc):
    
    for i in range(len(doc)):
        
        doc[i] = give_emoji_free_text(doc[i])
        
        #Tokenize with replacement
        doc[i] = tknz.tokenize(replacer.replace(doc[i]))
        
        #Filter stopwords, punctuations, and lowercase
        doc[i] = [w.lower() for w in doc[i] if w not in punc and w not in ALL_STOP_WORDS]
    
        # Bigram
        doc[i] = bigram_mod[doc[i]]
        
        #lemmatizer = WordNetLemmatizer()
        
        doc[i] = lemmatization(doc[i], allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
        
    return doc

In [ ]:
tweets = normalize(tweets)

In [ ]:
tweets[0] 

['speak',
 'event',
 'health',
 'care',
 'reform',
 'morning',
 'live',
 'tweet',
 'allow',
 'do',
 'happen',
 'true',
 'cover',
 'hard',
 'gawk',
 'building',
 'door',
 'son',
 'build',
 'ft',
 'replica',
 'backyard',
 'key',
 'question',
 'hit',
 'plan',
 'year',
 'hit',
 'plan',
 'year',
 'read',
 'ask',
 'goal',
 'orderly',
 'transition',
 'disruption',
 'generation',
 'family',
 'speak',
 'branch',
 'executive',
 'order',
 'specific',
 'action',
 'orderly',
 'transition',
 'massive',
 'disruption',
 'go',
 'motorcade',
 'parade',
 'protester',
 'chant',
 'be',
 'really',
 'motorcade',
 'head',
 'protester',
 'many',
 'slice',
 'read',
 'work',
 'solution',
 'trade',
 'off',
 'view',
 'queue',
 '7th',
 'office',
 'chant',
 'singing',
 'be',
 'scene',
 'start',
 'spread',
 'news',
 'bring',
 'quick',
 'thought',
 'ask',
 'how',
 'view',
 'ground',
 'poster',
 'reference',
 'interview',
 'continue',
 'press',
 'drug',
 'rebate',
 'classification',
 'be',
 'fallout',
 'happiness',
 's

In [ ]:
joint_tweets = [','.join(t) for t in tweets]

In [ ]:
pd.DataFrame(joint_tweets, columns=['Tweets']).to_csv('processed_tweets.csv')

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87197226-98be-42b2-8527-389082831299' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>